In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

In [49]:
df_flow = pd.DataFrame(
[[pd.Timestamp('2012-05-12'),1,-50],
[pd.Timestamp('2013-09-04'),1,100],
[pd.Timestamp('2014-05-05'),1,300],
[pd.Timestamp('2013-09-04'),2,-700],
[pd.Timestamp('2015-05-12'),2,1000],
[pd.Timestamp('2012-04-04'),3,100],
[pd.Timestamp('2013-05-12'),3,-50],
[pd.Timestamp('2013-09-04'),4,-60]],
columns= ['DATE','Investment','Flow'])
df_flow.sort_values('DATE', inplace=True)
df_flow

,DATE,Investment,Flow
5,2012-04-04,3,100
0,2012-05-12,1,-50
6,2013-05-12,3,-50
1,2013-09-04,1,100
3,2013-09-04,2,-700
7,2013-09-04,4,-60
2,2014-05-05,1,300
4,2015-05-12,2,1000


In [35]:
df_stock = pd.DataFrame(
[[pd.Timestamp('2012-09-05'),1,400],
[pd.Timestamp('2014-05-05'),1,600],
[pd.Timestamp('2014-05-05'),2,300],
[pd.Timestamp('2013-09-04'),2,800],
[pd.Timestamp('2012-09-14'),3,1000],
[pd.Timestamp('2013-09-05'),4,6000]],
columns=['DATE', 'Investment','Stock'])
df_stock.sort_values('DATE', inplace=True)
df_stock

,DATE,Investment,Stock
0,2012-09-05,1,400
4,2012-09-14,3,1000
3,2013-09-04,2,800
5,2013-09-05,4,6000
1,2014-05-05,1,600
2,2014-05-05,2,300


In [148]:
dft = pd.merge_asof(df_flow.rename(columns={'DATE':'DateFlow'}),
    df_stock.rename(columns={'DATE':'DateStock'}),
               left_on='DateFlow',
               right_on='DateStock',              
               by='Investment')
dft

,DateFlow,Investment,Flow,DateStock,Stock
0,2012-04-04,3,100,NaT,NaN
1,2012-05-12,1,-50,NaT,NaN
2,2013-05-12,3,-50,2012-09-14,1000.0
3,2013-09-04,1,100,2012-09-05,400.0
4,2013-09-04,2,-700,2013-09-04,800.0
5,2013-09-04,4,-60,NaT,NaN
6,2014-05-05,1,300,2014-05-05,600.0
7,2015-05-12,2,1000,2014-05-05,300.0


In [159]:
dftt = dft.melt(id_vars=['Investment','DateStock', 'DateFlow'],
               value_name='Flow+Stock').sort_values('Investment')
#dftt['DateStock'] = np.where(dftt.variable.eq('Stock'),dftt.DateStock, dftt.DateStock)
dftt.head()

,Investment,DateStock,DateFlow,variable,Flow+Stock
1,1,NaT,2012-05-12,Flow,-50.0
3,1,2012-09-05,2013-09-04,Flow,100.0
6,1,2014-05-05,2014-05-05,Flow,300.0
9,1,NaT,2012-05-12,Stock,NaN
11,1,2012-09-05,2013-09-04,Stock,400.0


In [162]:
dftt = dftt[['DateFlow','Investment', 'Flow+Stock']].rename(columns={'DateStock':'DATE'}).reset_index(drop=True)
dftt.dropna().head(6)

,DateFlow,Investment,Flow+Stock
0,2012-05-12,1,-50.0
1,2013-09-04,1,100.0
2,2014-05-05,1,300.0
4,2013-09-04,1,400.0
5,2014-05-05,1,600.0
6,2013-09-04,2,-700.0


In [ ]:
dftt

In [101]:
dftt = dft.melt(id_vars=['Investment','DateStock', 'DateFlow'],
               value_name='Flow+Stock').sort_values('Investment')
dftt['DateStock'] = np.where(dftt.variable.eq('Flow'),dftt.DateFlow, dftt.DateStock)
dftt = dftt[['DateStock', 'Investment', 'Flow+Stock']].rename(columns={'DateStock':'DATE'}).reset_index(drop=True)
dftt.tail()

,DATE,Investment,Flow+Stock
7,2013-09-04,2,-700
8,2012-09-14,3,1000
9,2012-04-04,3,100
10,2013-09-05,4,6000
11,2013-09-04,4,-60


In [115]:
uniqueInvestment = dftt.Investment.unique()
#create a data frame dictionary to store your data frames
DataFrameDict = {elem:('df'+str(elem)) for elem in uniqueInvestment}
DataFrameDict

'df1'

In [119]:
for uniqueInvestment in dftt.Investment.unique():
    print(dftt.loc[dftt.Investment == uniqueInvestment])


        DATE  Investment  Flow+Stock
0 2012-09-05           1         400
1 2014-05-05           1         600
2 2012-05-12           1         -50
3 2014-05-05           1         300
        DATE  Investment  Flow+Stock
4 2013-09-04           2         800
5 2014-05-05           2         300
6 2013-09-04           2        -700
7 2013-09-04           2        -700
        DATE  Investment  Flow+Stock
8 2012-09-14           3        1000
9 2012-04-04           3         100
         DATE  Investment  Flow+Stock
10 2013-09-05           4        6000
11 2013-09-04           4         -60


In [118]:
'1'

'1'

In [136]:
df_flow = pd.DataFrame(
[[pd.Timestamp('2012-05-12'),1,-50],
[pd.Timestamp('2013-09-04'),1,100],
[pd.Timestamp('2014-05-05'),1,300],
[pd.Timestamp('2013-09-04'),2,-700],
[pd.Timestamp('2015-05-12'),2,1000],
[pd.Timestamp('2012-04-04'),3,100],
[pd.Timestamp('2013-05-12'),3,-50],
[pd.Timestamp('2013-09-04'),4,-60]],
columns= ['DATE','Investment','Flow'])
df_flow.sort_values('DATE', inplace=True)
df_flow

,DATE,Investment,Flow
5,2012-04-04,3,100
0,2012-05-12,1,-50
6,2013-05-12,3,-50
1,2013-09-04,1,100
3,2013-09-04,2,-700
7,2013-09-04,4,-60
2,2014-05-05,1,300
4,2015-05-12,2,1000


In [137]:
df_stock = pd.DataFrame(
[[pd.Timestamp('2012-09-05'),1,400],
[pd.Timestamp('2014-05-05'),1,600],
[pd.Timestamp('2014-05-05'),2,300],
[pd.Timestamp('2013-09-04'),2,800],
[pd.Timestamp('2012-09-14'),3,1000],
[pd.Timestamp('2013-09-05'),4,6000]],
columns=['DATE', 'Investment','Stock'])
df_stock.sort_values('DATE', inplace=True)
df_stock

,DATE,Investment,Stock
0,2012-09-05,1,400
4,2012-09-14,3,1000
3,2013-09-04,2,800
5,2013-09-05,4,6000
1,2014-05-05,1,600
2,2014-05-05,2,300


In [138]:
df_m = pd.merge_asof(df_stock.rename(columns={'DATE':'DateStock'}),
               df_flow.rename(columns={'DATE':'DateFlow'}),
               left_on='DateStock',
               right_on='DateFlow',              
               by='Investment')
df_m

,DateStock,Investment,Stock,DateFlow,Flow
0,2012-09-05,1,400,2012-05-12,-50
1,2012-09-14,3,1000,2012-04-04,100
2,2013-09-04,2,800,2013-09-04,-700
3,2013-09-05,4,6000,2013-09-04,-60
4,2014-05-05,1,600,2014-05-05,300
5,2014-05-05,2,300,2013-09-04,-700


In [134]:
df_m = df_m.melt(id_vars=['Investment','DateStock', 'DateFlow'],
               value_name='Flow+Stock').sort_values('Investment')
df_m['DateStock'] = np.where(df_m.variable.eq('Flow'),df_m.DateFlow, df_m.DateStock)
df_m = df_m[['DateStock', 'Investment', 'Flow+Stock']].rename(columns={'DateStock':'DATE'}).reset_index(drop=True)


,DATE,Investment,Flow+Stock
7,2013-09-04,2,-700
8,2012-09-14,3,1000
9,2012-04-04,3,100
10,2013-09-05,4,6000
11,2013-09-04,4,-60


In [185]:
df_n = df_m

In [186]:
for uniqueInvestment in df_n.Investment.unique():
    print(df_n.loc[df_n.Investment == uniqueInvestment])

   DateStock  Investment  Stock   DateFlow  Flow
0 2012-09-05           1    400 2012-05-12   -50
4 2014-05-05           1    600 2014-05-05   300
   DateStock  Investment  Stock   DateFlow  Flow
1 2012-09-14           3   1000 2012-04-04   100
   DateStock  Investment  Stock   DateFlow  Flow
2 2013-09-04           2    800 2013-09-04  -700
5 2014-05-05           2    300 2013-09-04  -700
   DateStock  Investment  Stock   DateFlow  Flow
3 2013-09-05           4   6000 2013-09-04   -60


In [178]:
df = pd.DataFrame(
[[1, pd.Timestamp('2012-03-04')],
[1, pd.Timestamp('2013-04-15')],
[1, pd.Timestamp('2019-01-09')],
[2, pd.Timestamp('2013-04-09')],
[2, pd.Timestamp('2016-01-01')],
[2, pd.Timestamp('2018-05-09')]],
 columns=['ID','DATE']   
)
df

,ID,DATE
0,1,2012-03-04
1,1,2013-04-15
2,1,2019-01-09
3,2,2013-04-09
4,2,2016-01-01
5,2,2018-05-09


In [179]:
df.groupby('ID').agg(list).reset_index()

,ID,DATE
0,1,"[2012-03-04 00:00:00, 2013-04-15 00:00:00, 201..."
1,2,"[2013-04-09 00:00:00, 2016-01-01 00:00:00, 201..."


In [182]:
df = pd.DataFrame({"Col1": [10, 20, 15, 30, 45],
                  "Col2": [13, 23, 18, 33, 48],
                  "Col3": [17, 27, 22, 37, 52]},
                  index=pd.date_range("11:00", "13:00", freq="30min"))
df.resample('10T').asfreq()

,Col1,Col2,Col3
2021-04-22 11:00:00,10.0,13.0,17.0
2021-04-22 11:10:00,NaN,NaN,NaN
2021-04-22 11:20:00,NaN,NaN,NaN
2021-04-22 11:30:00,20.0,23.0,27.0
2021-04-22 11:40:00,NaN,NaN,NaN
2021-04-22 11:50:00,NaN,NaN,NaN
2021-04-22 12:00:00,15.0,18.0,22.0
2021-04-22 12:10:00,NaN,NaN,NaN
2021-04-22 12:20:00,NaN,NaN,NaN
2021-04-22 12:30:00,30.0,33.0,37.0


In [219]:
import pandas as pd

df = pd.DataFrame({"Point": ["A", "B", "C"], "x": [10, 20, 30], "y": [1, 2, 3]})

df["XX"] = 1

result = (
    df.merge(df, on="XX")
    .assign(distance=lambda d: (d["x_x"] - d["x_y"]) ** 2 + (d["y_x"] - d["y_y"]) ** 2)
    .loc[:, ["Point_x", "Point_y", "distance"]]
    .pivot(index="Point_x", columns="Point_y")
)
result

distance          
Point_y        A    B    C
Point_x                   
A              0  101  404
B            101    0  101
C            404  101    0

In [216]:
import pandas as pd

df = pd.DataFrame({"Point": ["A", "B", "C"], "x": [10, 20, 30], "y": [1, 2, 3]})

df["XX"] = 1

df

,Point,x,y,XX
0,A,10,1,1
1,B,20,2,1
2,C,30,3,1


In [217]:
df.merge(df, on="XX")
    #.assign(distance=lambda d: (d["x_x"] - d["x_y"]) ** 2 + (d["y_x"] - d["y_y"]) ** 2)
    #.loc[:, ["Point_x", "Point_y", "distance"]]
    #.pivot(index="Point_x", columns="Point_y")


,Point_x,x_x,y_x,XX,Point_y,x_y,y_y
0,A,10,1,1,A,10,1
1,A,10,1,1,B,20,2
2,A,10,1,1,C,30,3
3,B,20,2,1,A,10,1
4,B,20,2,1,B,20,2
5,B,20,2,1,C,30,3
6,C,30,3,1,A,10,1
7,C,30,3,1,B,20,2
8,C,30,3,1,C,30,3


In [406]:
df1 = pd.DataFrame({
    "key1": ['K', 'K', 'A1', 'A2', 'B1', 'B9', 'C3'],
    "key2": ['a5', 'a4', 'a7', 'a9', 'b2', 'b8', 'c1'],
    "Value1": ['apple', 'guava', 'kiwi', 'grape', 'banana', 'peach', 'berry'],
})
df2 = pd.DataFrame({
    "key1": ['K', 'A1', 'A3', 'B1', 'C2', 'C3'],
    "key2": ['a9', 'a7', 'a9', 'b2', 'c7', 'c1'],
    "Value2": ['apple', 'kiwi', 'grape', 'banana', 'guava', 'orange'],
})
df2

,key1,key2,Value2
0,K,a9,apple
1,A1,a7,kiwi
2,A3,a9,grape
3,B1,b2,banana
4,C2,c7,guava
5,C3,c1,orange


In [430]:
df1.merge(df2, on=['key1', 'key2'], how='outer').merge(df1, how='left')

,key1,key2,Value1,Value2
0,K,a5,apple,NaN
1,K,a4,guava,NaN
2,A1,a7,kiwi,kiwi
3,A2,a9,grape,NaN
4,B1,b2,banana,banana
5,B9,b8,peach,NaN
6,C3,c1,berry,orange
7,K,a9,NaN,apple
8,A3,a9,NaN,grape
9,C2,c7,NaN,guava


In [463]:
df = pd.DataFrame([
['Peter','en',1],
['Petrus','latin',1],
['Peter','de',1],
['Victor','en',2],
['Victor','latin',2],
['Claudius','latin',3],
['Claudio','spanish',3]],
columns=['name','language','gr_id']
)
df

,name,language,gr_id
0,Peter,en,1
1,Petrus,latin,1
2,Peter,de,1
3,Victor,en,2
4,Victor,latin,2
5,Claudius,latin,3
6,Claudio,spanish,3


In [500]:
df1 = df.pivot(index='gr_id', columns='language')
np.where(df2.name.isin(df1.name),1,2)

array([2, 2, 2])

In [501]:
df1.name

language,de,en,latin,spanish
gr_id,,,,
1,Peter,Peter,Petrus,NaN
2,NaN,Victor,Victor,NaN
3,NaN,NaN,Claudius,Claudio


In [526]:
df.pivot(columns=['gr_id'], index='language').name

gr_id,1,2,3
language,,,
de,Peter,NaN,NaN
en,Peter,Victor,NaN
latin,Petrus,Victor,Claudius
spanish,NaN,NaN,Claudio


In [505]:
from io import StringIO
import csv
data = """app_id_dummy,alert_type,timestamp,session_id
test,active,1618870097991,63f3eded98
test,active,1618869937038,ea86303028
test,active,1618869423974,55e809427a"""

dia = csv.excel()
dia.quoting = csv.QUOTE_NONE
pd.read_csv(StringIO(data), dialect=dia)

,app_id_dummy,alert_type,timestamp,session_id
0,test,active,1618870097991,63f3eded98
1,test,active,1618869937038,ea86303028
2,test,active,1618869423974,55e809427a


In [57]:
df = pd.DataFrame([
[1, 'mcma',123],
[1,'mcms',124],
[1,'mcda',125],
[1,'mcds',126],
[2,'cic',127],
[2,'cmc',128]],
columns=['id','type','ip'])

In [59]:
(df.assign(ip = df['ip'].where(df['type'].eq('mcma')))
        .groupby('id')
        .agg(type=('type',list), ip=('ip','first'))
        .reset_index())

,id,type,ip
0,1,"[mcma, mcms, mcda, mcds]",123.0
1,2,"[cic, cmc]",NaN


In [61]:
df.groupby('id').agg(list)

,type,ip
id,,
1,"[mcma, mcms, mcda, mcds]","[123, 124, 125, 126]"
2,"[cic, cmc]","[127, 128]"


In [94]:
df = pd.DataFrame({'id':['x1', 'x1', 'x1', 'x2', 'x2', 'x2', 'x2']
   ,  'beg':['2021-01-01 00:00:00',
   '2021-02-03 00:00:00','2021-02-04 00:00:00','2021-02-05 00:00:00',
   '2021-02-06 00:00:00','2021-03-05 00:00:00','2021-04-01 00:00:00'],
      'end':['2021-01-02 00:00:00 ',
   '2021-02-03 12:00:00','2021-02-04 10:00:00','2021-02-05 10:00:00',
   '2021-02-06 10:00:00','2021-03-07 10:00:00','2021-05-08 00:00:00']})
df

,id,beg,end
0,x1,2021-01-01 00:00:00,2021-01-02 00:00:00
1,x1,2021-02-03 00:00:00,2021-02-03 12:00:00
2,x1,2021-02-04 00:00:00,2021-02-04 10:00:00
3,x2,2021-02-05 00:00:00,2021-02-05 10:00:00
4,x2,2021-02-06 00:00:00,2021-02-06 10:00:00
5,x2,2021-03-05 00:00:00,2021-03-07 10:00:00
6,x2,2021-04-01 00:00:00,2021-05-08 00:00:00


In [96]:
df = pd.concat([pd.Series(r.id,pd.date_range(r.beg, r.end, freq='H')) for r in df.itertuples()]).reset_index()
df.columns=['date','id']

df = df.groupby(['id', df['date'].dt.strftime('%m/%Y')]).size().reset_index(name='count')
print (df)

   id     date  count
0  x1  01/2021     25
1  x1  02/2021     24
2  x2  02/2021     22
3  x2  03/2021     59
4  x2  04/2021    720
5  x2  05/2021    169


In [2]:
series_1 = [[19,"a"],[20,"d"],[31,"d"],[31,"c"],[51,"d"]]
a_df = pd.DataFrame(series_1)
a_df

,0,1
0,19,a
1,20,d
2,31,d
3,31,c
4,51,d


In [3]:
cols = ["a","c","d"]
series_3 = [1,0,0,
           0,0,1,
           0,1,1,
           0,0,1]
np_series = np.array(series_3).reshape(4,3)
c_df = pd.DataFrame(np_series,index = [19,20,31,51],columns=cols)
c_df

,a,c,d
19,1,0,0
20,0,0,1
31,0,1,1
51,0,0,1


In [25]:
a_df.pivot_table(values=0, index=a_df.index, columns=1, aggfunc='first')

1,a,c,d
0,19.0,NaN,NaN
1,NaN,NaN,20.0
2,NaN,NaN,31.0
3,NaN,31.0,NaN
4,NaN,NaN,51.0


In [36]:
a_df.set_index(0).stack().to_frame().reset_index(level=1)

,level_1,0
0,,
19,1,a
20,1,d
31,1,d
31,1,c
51,1,d


In [110]:
a_df.set_index(0).stack().to_frame().reset_index(level=1).rename(
    columns={(0):'col'}).pivot_table(
    columns='col', values='level_1', index=a_df)

,level_1,col
0,,
19,1,a
20,1,d
31,1,d
31,1,c
51,1,d


In [111]:
data = """Name,StartPoint,EndPoint,isDelivered,Customer
A,1,4,0,C1
A,1,4,0,C1
A,2,5,1,C1
A,3,5,0,C1
A,3,6,0,C1
A,3,6,1,C1
B,1,4,0,C2
B,1,4,0,C2
B,2,5,1,C2
B,3,5,1,C2
B,3,6,1,C2
B,3,8,0,C2
B,3,8,1,C2"""

df=pd.read_csv(StringIO(data))
df

,Name,StartPoint,EndPoint,isDelivered,Customer
0,A,1,4,0,C1
1,A,1,4,0,C1
2,A,2,5,1,C1
3,A,3,5,0,C1
4,A,3,6,0,C1
5,A,3,6,1,C1
6,B,1,4,0,C2
7,B,1,4,0,C2
8,B,2,5,1,C2
9,B,3,5,1,C2


In [126]:
df.groupby('Name')['StartPoint'].min()

Name
A    1
B    1
Name: StartPoint, dtype: int64

In [131]:
data = """[
   {
      "id":"abcd",
      "events":[
         {
            "description":"Liquidated",
            "time":"2020-12-18T12:00:00Z",
            "origin":{
               "name":"abc, GA",
               "code":" "
            },
            "destination":{
               "name":" ",
               "code":"SAV"
            },
            "remarks":"L 99",
            "invoice":"pqr",
            "code":"LIQ"
         },
         {
            "description":"Customs Monthly Stmt Paid",
            "time":"2020-02-26T12:00:00Z",
            "origin":{
               
            },
            "destination":{
               "name":"SAVANNAH, GA",
               "code":"SAV"
            },
            "remarks":"PM Stmt No 1720P0207Q",
            "invoice":"pqr",
            "code":"MSP"
         }
         ]
  },
  {
      "id":"xyz",
      "events":[
         {
            "description":"Liquidated",
            "time":"2020-12-18T12:00:00Z",
            "origin":{
               "name":"abc, GA",
               "code":" "
            },
            "destination":{
               "name":"abc",
               "code":""
            },
            "remarks":"L 99",
            "invoice":" ",
            "code":"LIQ"
         },
         {
            "description":"Customs Monthly Stmt Paid",
            "time":"2020-02-26T12:00:00Z",
            "origin":{
               
            },
            "destination":{
               "name":"SAVANNAH, GA",
               "code":"SAV"
            },
            "remarks":"PM Stmt No 1720P0207Q",
            "invoice":" ",
            "code":"MSP"
         }
         ]
}
]"""
df = pd.read_json(StringIO(data))
df.events.get(1)

[{'description': 'Liquidated',
  'time': '2020-12-18T12:00:00Z',
  'origin': {'name': 'abc, GA', 'code': ' '},
  'destination': {'name': 'abc', 'code': ''},
  'remarks': 'L 99',
  'invoice': ' ',
  'code': 'LIQ'},
 {'description': 'Customs Monthly Stmt Paid',
  'time': '2020-02-26T12:00:00Z',
  'origin': {},
  'destination': {'name': 'SAVANNAH, GA', 'code': 'SAV'},
  'remarks': 'PM Stmt No 1720P0207Q',
  'invoice': ' ',
  'code': 'MSP'}]

In [442]:
data="""Date,Tutor,Student
2021-04-12,310,112
2021-04-13,394,210
2021-04-14,357,3
2021-04-15,359,0
2021-04-16,392,0
2021-04-17,307,0
2021-04-18,335,0
2021-04-19,0,121"""
df=pd.read_csv(StringIO(data))
df = df.set_index('Date')
df = df.rename_axis('Name', axis=1)
df

Name,Tutor,Student
Date,,
2021-04-12,310,112
2021-04-13,394,210
2021-04-14,357,3
2021-04-15,359,0
2021-04-16,392,0
2021-04-17,307,0
2021-04-18,335,0
2021-04-19,0,121


In [446]:
df.iloc[:].apply(pd.to_timedelta, unit='s') 

Name,Tutor,Student
Date,,
2021-04-12,0 days 00:05:10,0 days 00:01:52
2021-04-13,0 days 00:06:34,0 days 00:03:30
2021-04-14,0 days 00:05:57,0 days 00:00:03
2021-04-15,0 days 00:05:59,0 days 00:00:00
2021-04-16,0 days 00:06:32,0 days 00:00:00
2021-04-17,0 days 00:05:07,0 days 00:00:00
2021-04-18,0 days 00:05:35,0 days 00:00:00
2021-04-19,0 days 00:00:00,0 days 00:02:01


In [158]:
pd.to_timedelta(df.Tutor.astype(str).add_suffix('s'))

Date
2021-04-12s   0 days 00:00:00.000000310
2021-04-13s   0 days 00:00:00.000000394
2021-04-14s   0 days 00:00:00.000000357
2021-04-15s   0 days 00:00:00.000000359
2021-04-16s   0 days 00:00:00.000000392
2021-04-17s   0 days 00:00:00.000000307
2021-04-18s   0 days 00:00:00.000000335
2021-04-19s             0 days 00:00:00
Name: Tutor, dtype: timedelta64[ns]

In [159]:
df.iloc[:].apply(pd.to_timedelta)

Name,Tutor,Student
Date,,
2021-04-12,0 days 00:00:00.000000310,0 days 00:00:00.000000112
2021-04-13,0 days 00:00:00.000000394,0 days 00:00:00.000000210
2021-04-14,0 days 00:00:00.000000357,0 days 00:00:00.000000003
2021-04-15,0 days 00:00:00.000000359,0 days 00:00:00
2021-04-16,0 days 00:00:00.000000392,0 days 00:00:00
2021-04-17,0 days 00:00:00.000000307,0 days 00:00:00
2021-04-18,0 days 00:00:00.000000335,0 days 00:00:00
2021-04-19,0 days 00:00:00,0 days 00:00:00.000000121


In [163]:
df.iloc[:].apply(pd.to_timedelta)

Name,Tutor,Student
Date,,
2021-04-12,0 days 00:00:00.000000310,0 days 00:00:00.000000112
2021-04-13,0 days 00:00:00.000000394,0 days 00:00:00.000000210
2021-04-14,0 days 00:00:00.000000357,0 days 00:00:00.000000003
2021-04-15,0 days 00:00:00.000000359,0 days 00:00:00
2021-04-16,0 days 00:00:00.000000392,0 days 00:00:00
2021-04-17,0 days 00:00:00.000000307,0 days 00:00:00
2021-04-18,0 days 00:00:00.000000335,0 days 00:00:00
2021-04-19,0 days 00:00:00,0 days 00:00:00.000000121


In [165]:
df.iloc[:].apply(pd.to_timedelta) 

Name,Tutor,Student
Date,,
2021-04-12,0 days 00:00:00.000000310,0 days 00:00:00.000000112
2021-04-13,0 days 00:00:00.000000394,0 days 00:00:00.000000210
2021-04-14,0 days 00:00:00.000000357,0 days 00:00:00.000000003
2021-04-15,0 days 00:00:00.000000359,0 days 00:00:00
2021-04-16,0 days 00:00:00.000000392,0 days 00:00:00
2021-04-17,0 days 00:00:00.000000307,0 days 00:00:00
2021-04-18,0 days 00:00:00.000000335,0 days 00:00:00
2021-04-19,0 days 00:00:00,0 days 00:00:00.000000121


In [193]:
df = DataFrame({'a':['x','x','y','y'], 'b':['1000','2000','3000','4000'],
               'c':[100,30,300,20]})
df

,a,b,c
0,x,1000,100
1,x,2000,30
2,y,3000,300
3,y,4000,20


In [194]:
df.set_index(['a','b']).reset_index().groupby('a', as_index=False).last()

,a,b,c
0,x,2000,30
1,y,4000,20


In [196]:
df1 = pd.DataFrame({'ID': ['AAA','AAA', 'AAA','BBB','CCC'], 'Buydate': ['2017-04-13', '2019-12-31', '2019-03-05', '2018-11-04', '2019-12-31' ], 'Quantity': [100.00, 2000.00, 385.95, 214514.00, 63205.00]}) 
df1 = df1.sort_values('Buydate')
#df1 = df1.set_index(['ID', 'Buydate']).reset_index()
df1['Buydate'] = pd.to_datetime(df1.Buydate)
df1

KeyError: "None of ['ROUGE_L'] are in the columns"

In [242]:
import random
# input datast
length = 10
ts = pd.date_range(start='2021/01/01 00:00', periods=length, freq='1h')
random.seed(1)
val = random.sample(range(1, 10+length), length)
df = pd.DataFrame({'val' : val}, index=ts)

# groupby custom datetimeindex
key_ts = [ts[i] for i in [1,3,7]]
df.loc[key_ts, 'id'] = range(len(key_ts))
df['id'] = df['id'].ffill()
df

,val,id
2021-01-01 00:00:00,5,NaN
2021-01-01 01:00:00,3,0.0
2021-01-01 02:00:00,9,0.0
2021-01-01 03:00:00,4,1.0
2021-01-01 04:00:00,8,1.0
2021-01-01 05:00:00,13,1.0
2021-01-01 06:00:00,15,1.0
2021-01-01 07:00:00,14,2.0
2021-01-01 08:00:00,11,2.0
2021-01-01 09:00:00,7,2.0


In [243]:
df['id'] = df.id.astype('category')

In [245]:
%timeit df.groupby('id')['val'].cumsum()

580 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [236]:
%timeit df.groupby('id')['val'].cumsum()

671 µs ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [209]:
df.groupby('id')['val'].cumsum()

2021-01-01 00:00:00    -1
2021-01-01 01:00:00     3
2021-01-01 02:00:00    12
2021-01-01 03:00:00     4
2021-01-01 04:00:00    12
2021-01-01 05:00:00    25
2021-01-01 06:00:00    40
2021-01-01 07:00:00    14
2021-01-01 08:00:00    25
2021-01-01 09:00:00    32
Freq: H, Name: val, dtype: int64

In [226]:
df['cumsum'] = df.groupby('id', sort=False)['val'].cumsum()
df

,val,id,cumsum
2021-01-01 00:00:00,5,NaN,0
2021-01-01 01:00:00,3,0.0,3
2021-01-01 02:00:00,9,0.0,12
2021-01-01 03:00:00,4,1.0,4
2021-01-01 04:00:00,8,1.0,12
2021-01-01 05:00:00,13,1.0,25
2021-01-01 06:00:00,15,1.0,40
2021-01-01 07:00:00,14,2.0,14
2021-01-01 08:00:00,11,2.0,25
2021-01-01 09:00:00,7,2.0,32


In [313]:
df = pd.DataFrame({
    'ID':np.repeat(101,6),
    'Currency':np.repeat('INR',6),
    'Val1':np.arange(1,7),
    'Val2':np.arange(7,13),
    'Month':pd.period_range(start='2020-10', end='2021-03', freq='M')[::-1]
})
df

,ID,Currency,Val1,Val2,Month
0,101,INR,1,7,2021-03
1,101,INR,2,8,2021-02
2,101,INR,3,9,2021-01
3,101,INR,4,10,2020-12
4,101,INR,5,11,2020-11
5,101,INR,6,12,2020-10


In [276]:
pd.pivot(df.set_index(['ID','Currency'])

Month                                             
val2                    7        8        9        10       11       12
ID  Currency val1                                                      
101 INR      1     2021-03      NaT      NaT      NaT      NaT      NaT
             2         NaT  2021-02      NaT      NaT      NaT      NaT
             3         NaT      NaT  2021-01      NaT      NaT      NaT
             4         NaT      NaT      NaT  2020-12      NaT      NaT
             5         NaT      NaT      NaT      NaT  2020-11      NaT
             6         NaT      NaT      NaT      NaT      NaT  2020-10

In [254]:
list(reversed(pd.period_range(start='2020-10', end='2021-03', freq='M')))

[Period('2021-03', 'M'),
 Period('2021-02', 'M'),
 Period('2021-01', 'M'),
 Period('2020-12', 'M'),
 Period('2020-11', 'M'),
 Period('2020-10', 'M')]

In [311]:
df.set_index(['ID','Currency','Month']).stack().unstack(-2).reset_index()

Month,ID,Currency,level_2,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03
0,101,INR,val1,6,5,4,3,2,1
1,101,INR,val2,12,11,10,9,8,7


In [330]:
pd.pivot(
        df,
        index=["ID", "Currency"],
        columns="Month",
        values=["Val1", "Val2"],
    ).stack(0)

Month              2020-10  2020-11  2020-12  2021-01  2021-02  2021-03
ID  Currency                                                           
101 INR      Val1        6        5        4        3        2        1
             Val2       12       11       10        9        8        7

In [334]:
data = """id,grouping_val
1,a
2,a
3,b
4,b"""
df = pd.read_csv(StringIO(data))
df['data'] = [[[0, 1], [1, 0]], [[1, 0], [0, 1]], [[2, 0], [3, 0]],
             [[0, 4], [4, 5]]]
df

,id,grouping_val,data
0,1,a,"[[0, 1], [1, 0]]"
1,2,a,"[[1, 0], [0, 1]]"
2,3,b,"[[2, 0], [3, 0]]"
3,4,b,"[[0, 4], [4, 5]]"


In [344]:
np.flat(df.data)

AttributeError: module 'numpy' has no attribute 'flat'

In [376]:
df = pd.DataFrame({'date': ['2017-04-13', '2019-5-31', '2019-03-05', '2018-3-04', '2019-7-31' ], 'Quantity': [100.00, 2000.00, 385.95, 214514.00, 63205.00]}) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      5 non-null      object 
 1   Quantity  5 non-null      float64
dtypes: float64(1), object(1)
memory usage: 208.0+ bytes


In [370]:
df['date']=pd.to_datetime(df.date)
#df = df.set_index('date')


In [371]:
#df.loc['2013-01-01':'2019-12-31']

In [372]:
mask = (df['date'] > '2017-04-13') & (df['date'] <= '2019-04-13')
df.loc[mask]

,date,Quantity
2,2019-03-05,385.95
3,2018-03-04,214514.00


In [378]:
mask = (df['Date'] > '01-01-2013') & (df['Date'] <= '31-12-2018')
df.loc[mask]

KeyError: 'Date'

In [377]:
df[mask]

,date,Quantity
0,2017-04-13,100.0
3,2018-3-04,214514.0


In [393]:
dates=['2017-12-01','2017-12-05','2017-12-17','2017-12-28','2018-01-05','2018-01-09']
val=[5,2,3,6,1,7]
df = pd.DataFrame(list(zip(dates, val)),
               columns =['date', 'val'])
df['date'] = pd.to_datetime(df.date)
df

,date,val
0,2017-12-01,5
1,2017-12-05,2
2,2017-12-17,3
3,2017-12-28,6
4,2018-01-05,1
5,2018-01-09,7


In [422]:
df['week'] = 1+(df['date']-df.loc[0,'date']).dt.days // 7
df

,date,val,week
0,2017-12-01,5,1
1,2017-12-05,2,1
2,2017-12-17,3,3
3,2017-12-28,6,4
4,2018-01-05,1,6
5,2018-01-09,7,6


In [440]:
data = """created,id
1601820360,10
1601820365,11"""
df =pd.read_csv(StringIO(data))
df

,created,id
0,1601820360,10
1,1601820365,11


In [441]:
df['created'] = pd.to_datetime(df.created, unit='s')
df

,created,id
0,2020-10-04 14:06:00,10
1,2020-10-04 14:06:05,11


In [469]:
df = DataFrame({
    'a':['x','x','y','x','y','z'],
    'b':np.arange(1,7),
    'c':np.arange(1,7)
})



In [470]:
df.groupby('a', as_index=False)[['b','c']].count()


,a,b,c
0,x,3,3
1,y,2,2
2,z,1,1


In [477]:
data = """created_at, value1,value2,value3
2021-04-25 11:38:33,1,1,5
2021-04-25 11:38:47,4,3,6
2021-04-25 11:39:36,1,1,8 
2021-04-25 11:39:47,6,5,5 
2021-04-25 11:40:50,8,7,3"""
df = pd.read_csv(StringIO(data))
df['created_at'] = pd.to_datetime(df.created_at)
df

,created_at,value1,value2,value3
0,2021-04-25 11:38:33,1,1,5
1,2021-04-25 11:38:47,4,3,6
2,2021-04-25 11:39:36,1,1,8
3,2021-04-25 11:39:47,6,5,5
4,2021-04-25 11:40:50,8,7,3


In [478]:
pd.DataFrame(df.groupby([pd.Grouper(key='created_at', freq='2Min')]).mean())

,value1,value2,value3
created_at,,,
2021-04-25 11:38:00,3.0,2.5,6.0
2021-04-25 11:40:00,8.0,7.0,3.0


In [481]:
data="""date,id
2020-10-04 14:06:00,10.0
2020-10-04 14:06:05,11.0 
2020-10-05 12:28:58,12.0
2020-10-05 12:29:16,13.0
2020-10-05 12:29:31,14.0"""
df = pd.read_csv(StringIO(data))
df['date'] = pd.to_datetime(df.date)
df

,date,id
0,2020-10-04 14:06:00,10.0
1,2020-10-04 14:06:05,11.0
2,2020-10-05 12:28:58,12.0
3,2020-10-05 12:29:16,13.0
4,2020-10-05 12:29:31,14.0


In [487]:
pd.DataFrame(df.groupby([pd.Grouper(key='date', freq='15Min')]).sum()).reset_index().head()

,date,id
0,2020-10-04 14:00:00,21.0
1,2020-10-04 14:15:00,0.0
2,2020-10-04 14:30:00,0.0
3,2020-10-04 14:45:00,0.0
4,2020-10-04 15:00:00,0.0


In [576]:
df = pd.DataFrame({"col_1": ["apple", "banana", "apple", "banana", "banana"],
                   "col_2": [1, 4, 8, 8, 6],
                   "col_3": [56, 4, 22, 1, 5]})
df

,col_1,col_2,col_3
0,apple,1,56
1,banana,4,4
2,apple,8,22
3,banana,8,1
4,banana,6,5


In [504]:
dfo = DataFrame({"col_1": ["apple", "banana"],
 "col_2": [[1, 8], [4, 8, 6]],
                 ] "col_3": [[56, 22], [4, 1, 5]],
 "count": [2,3]})
dfo

,col_1,col_2,col_3,count
0,apple,"[1, 8]","[56, 22]",2
1,banana,"[4, 8, 6]","[4, 1, 5]",3


In [564]:
df['count'] = df.groupby('col_1')['col_2'].transform('count')
df

,col_1,col_2,col_3,count
0,apple,1,56,2
1,banana,4,4,3
2,apple,8,22,2
3,banana,8,1,3
4,banana,6,5,3


In [586]:
%timeit df.groupby('col_1').agg({ 'col_2': lambda x: list(x),'col_3': lambda x: list(x),}).reset_index().join(df.groupby('col_1')['col_2'].transform('count').rename('count'))

6.11 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [587]:
%timeit df.groupby('col_1')[['col_2', 'col_3']].agg(list).join(df.groupby('col_1').size().rename('count'))

9.09 ms ± 217 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
